# Julia's Type System

Julia programs manipulate *values*, and every value has two parts: a *type* part and a data part. The type part answers the question "what kind of thing is this?", and the data part distinguishes one thing of a certain kind from every other thing of that kind.

## Part 1. DataType

In [12]:
typeof(3)

Int64

In this case the type is `Int64` and the data part is the bits `...0011`.

In Julia types are also values:

In [13]:
typeof(Int64)

DataType

In [14]:
typeof(DataType)

DataType

In fact, the identity `typeof(typeof(x)) === DataType` holds for all values in Julia. `DataType` is the backbone of the entire system. It does many jobs, which can be identified by looking inside a `DataType` object:

### DataType Job 1: A symbolic description

This consists of a name (which is mostly a string), and a vector of sub-components:

In [15]:
T = typeof(1+2im)

Complex{Int64}

In [17]:
T.name

Complex

In [18]:
T.parameters

svec(Int64)

### DataType Job 2: A nominal hierarchy of types

DataTypes form a tree of declared type relationships ("an x is-a y"):

In [19]:
T.super

Number

In [21]:
T.super.super  # `Any` is the built-in top of the hierarchy.

Any

In [26]:
Int.super.super.super.super.super

Any

### DataType Job 3: Describe the representation

In [27]:
T.types

svec(Int64, Int64)

In [28]:
T.name.names

svec(:re, :im)

In [29]:
T.size

16

In [30]:
T.mutable   # whether this was declared with `type` (vs. `immutable`)

false

In [31]:
T.abstract  # whether this was declared with `abstract`

false

In [32]:
T.ninitialized

2

In [33]:
T.layout

Ptr{Nothing} @0x00007fb7d6854e18

### Defining struct types


In [39]:
mutable struct MPoint
    x::Float64
    y::Float64
end

In [41]:
p = Point(1,"")

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [40]:
methods(Point)

# 2 methods for generic function "(::Type)":
[1] Point(x::Float64, y::Float64) in Main at In[34]:2
[2] Point(x, y) in Main at In[34]:2

In [1]:
struct Point
    x
    y
    Point(x) = new(x, x)
end

Point(x,y) = Point(x+y)

In [6]:
Point("hi","")

MethodError: MethodError: no method matching Point(::String, ::String)
Closest candidates are:
  Point(::Any) at In[1]:4

### Abstract vs. Concrete

`abstract` types can have declared subtypes, while concrete types can have instances. These are separated because if an `X` IS-A `Y`, and `Y` specifies a representation, then `X` had better have the same representation.

"car is-a vehicle" is correct because "vehicle" is an abstract concept that doesn't commit to any specifics. But if I tell you I'm giving you a Porsche, it needs to look like a Porsche.

A type `T` is concrete if there could be some value `x` such that `typeof(x) === T`. This is also sometimes called a "leaf type".

In [ ]:
abstract type PointLike end

# struct Point <: PointLike

## Part 2. Type parameters

Type parameters can be completely or partially specified:

In [8]:
Array{Int,2} <: Array{Int} <: Array

true

In [9]:
Array{Int}

Array{Int64,N} where N

In [11]:
[1] isa Array{Int}

true

In [13]:
[1]::Int

TypeError: TypeError: in typeassert, expected Int64, got Array{Int64,1}

In [ ]:
Array{Int,2}

A type is concrete (can have instances) if
    1. it is not declared `abstract`
    2. all parameters are specified

In [ ]:
[1] isa Array{Int,1}

In [14]:
[1] isa Array{Int}

true

In [15]:
[1] isa Array{Number}

false

In [ ]:
Int <: Number

Types with different *specified* parameters are just different, and have no subtype relationship. This is called *invariance*.

### Defining types with parameters

In [17]:
struct GenericPoint{T<:Real}
    x::T
    y::T
end

In [18]:
GenericPoint(1,2)

GenericPoint{Int64}(1, 2)

In [19]:
GenericPoint(1.0,2.0)

GenericPoint{Float64}(1.0, 2.0)

In [21]:
methods(GenericPoint)

# 1 method for generic function "(::Type)":
[1] (::Type{GenericPoint})(x::T, y::T) where T<:Real in Main at In[17]:2

In [25]:
GenericPoint{Int8}(1,1000)

InexactError: InexactError: trunc(Int8, 1000)

In [23]:
methods(GenericPoint{Int8})

# 1 method for generic function "(::Type)":
[1] (::Type{GenericPoint{T}})(x, y) where T<:Real in Main at In[17]:2

In [20]:
GenericPoint(1,2.0)

MethodError: MethodError: no method matching GenericPoint(::Int64, ::Float64)
Closest candidates are:
  GenericPoint(::T<:Real, !Matched::T<:Real) where T<:Real at In[17]:2

### Tuple types

In [27]:
typeof((1,2.0,""))

Tuple{Int64,Float64,String}

Very similar to other DataTypes, except
    1. Have no field names, only indices
    2. `T.parameters == T.types`
    3. Are always immutable
    4. Can have any number of fields

These factors conspire to make Tuples the only *covariant* types in Julia:

In [28]:
Tuple{Int} <: Tuple{Number}

true

A Tuple type is concrete iff all its field types are.

Tuple types can be abstract with respect to the number of elements. These are called variadic tuple types, or vararg types.

In [33]:
(1,1,1.0) isa Tuple{Int,Vararg{Int}}

false

Note that `Vararg` refers to the tail of a tuple type, and as such is not a first-class type itself. It only makes sense inside a Tuple type. This is a bit unfortunate.

The second parameter to `Vararg` is a length, which can also be either unspecified (as above), or specified:

In [35]:
Tuple{Int,Vararg{Int,20}}

NTuple{21,Int64}

## Part 3. Larger type domains

### Union types

A type can be thought of as a set of possible values. A type expresses *uncertainty* about which value we have. You can do set operations on them.

In [36]:
Union{Int64,Float64}

Union{Float64, Int64}

In [38]:
1.0 isa Union{Int64,Float64}

true

In [39]:
Int64 <: Number

true

In [40]:
Int64 <: Union{Int64,Float64}

true

In [41]:
Union{Int,String} <: Union{Int,String,Float32}

true

In [42]:
typeintersect(Union{Int,String}, Union{Int,String,Float32})

Union{Int64, String}

Union types naturally lend themselves to missing data.

In [43]:
data = [1.1, missing, 3.2, missing, 5.7, 0.4]

6-element Array{Union{Missing, Float64},1}:
 1.1     
  missing
 3.2     
  missing
 5.7     
 0.4     

### UnionAll types

This is an *iterated union* of types.

`Array{T,1} where T<:Integer`

Means "the union of all types of the form Array{T,1} where T is a subtype of Integer".

This expresses uncertainty about the value of a parameter.

* Since this kind of type introduces *variables*, its expressive power is (probably) equivalent to quantified boolean formulas.
* Requires a quantified-SAT solver in the compiler.
* Under common assumptions, harder than NP-complete.

In [44]:
# this definition is in the Base library
Vector = Array{T,1} where T

Array{T,1} where T

In [45]:
Vector{Int}

Array{Int64,1}

These are used to express "unspecified parameters".

These also describe methods with "method parameters" (or "static parameters"):

In [46]:
func(a::Array{T,1}) where {T<:Integer} = T

func (generic function with 1 method)

In [47]:
func([0x00])

UInt8

In [49]:
func([1])

Int64

#### Question

What is the difference between

`Vector{Vector{T}} where T`

and

`Vector{Vector{T} where T}`?

Is one a subtype of the other?

In [50]:
function f(a::Array{<:Number,N}, xs::Vararg{Int,N}) where {N}
    N
end

f (generic function with 1 method)

In [51]:
f([1],1)

1

In [52]:
f([1],1,2)

MethodError: MethodError: no method matching f(::Array{Int64,1}, ::Int64, ::Int64)
Closest candidates are:
  f(::Array{#s1,N} where #s1<:Number, !Matched::Int64...) where N at In[50]:2

In [53]:
f(rand(2,2),1)

MethodError: MethodError: no method matching f(::Array{Float64,2}, ::Int64)
Closest candidates are:
  f(::Array{#s1,N} where #s1<:Number, !Matched::Int64...) where N at In[50]:2

In [54]:
f(rand(2,2),1,2)

2